In [1]:
import os
os.chdir('../../')

In [2]:
import pandas as pd
t1 = pd.read_feather('./experiments/8fdtxcm4'+'/predictions_test.feather')
t2 = pd.read_feather('./dataset/mimiciv/note/discharge.feather')

# Perform the join
merged_table = t1.merge(t2, left_on='_id', right_on='hadm_id', how='inner')

# Display the merged table
merged_table=merged_table[['_id','text','target']]

In [3]:
merged_table.iloc[200]

_id                                                27703599
text       \nName:  ___                   Unit No:   ___...
target    [Y83.8, T81.4XXA, Y92.9, J45.909, 02HV33Z, E66...
Name: 200, dtype: object

In [ ]:
# สมมติข้อความคุณอยู่ในตัวแปรนี้
raw_target = merged_table.iloc[200]['target']
formatted_target = ', '.join(f'"{code}"' for code in raw_target)
# สมมติข้อความคุณอยู่ในตัวแปรนี้
raw_text = merged_table.iloc[200]['text']

# แปลงเป็นบรรทัดใหม่ พร้อมลบช่องว่างหน้าหลังที่ไม่จำเป็น
formatted_text = raw_text.replace('\\n', '\n').strip()

In [ ]:
import pandas as pd
predictions_test = pd.read_feather('./experiments/8fdtxcm4'+'/predictions_test.feather')
id=predictions_test.iloc[200:201,-2:]['_id'].values[0]
target=predictions_test.iloc[200:201,-2:]
discharge = pd.read_feather('./dataset/mimiciv/note/discharge.feather')
discharge[discharge['hadm_id']==id]

In [ ]:
import torch

# Database tensor (14x3)
db = torch.tensor([[0.3124, 0.7484, 0.5882],
        [0.7315, 0.7216, 0.4280],
        [0.8297, 0.5152, 0.5855],
        [0.7019, 0.1183, 0.1639],
        [0.4908, 0.4098, 0.5747],
        [0.5028, 0.3644, 0.3118],
        [0.8006, 0.4837, 0.6474],
        [0.2512, 0.3838, 0.3375],
        [0.2173, 0.1948, 0.8814],
        [0.7787, 0.7503, 0.3902],
        [-0.7949, 0.7372, -0.7418],
        [0.7702, 0.7498, 0.3535],
        [0.3216, 0.1167, 0.6975],
        [0.3265, 0.5124, 0.6395]])

# Query Vectors (2x3)
qv = torch.tensor([[0.7461, 0.6752, 0.7194],
                   [0.7702, 0.7498, 0.3535]])
qv.shape

In [ ]:
import torch

tensor_data = torch.tensor([[[0.7949, 0.7372, 0.7418],
                             [0.4908, 0.4098, 0.5747],
                             [0.8006, 0.4837, 0.6474],
                             [0.8297, 0.5152, 0.5855],
                             [0.0000, 0.0000, 0.0000]],

                            [[0.4908, 0.4098, 0.5747],
                             [0.7949, 0.0000, 0.7418],
                             [0.0000, 0.0000, 0.0000],
                             [0.0000, 0.0000, 0.0000],
                             [0.0000, 0.0000, 0.0000]]])

# Step 1: Remove trailing zero rows
def remove_trailing_zeros(tensor):
    mask = ~(tensor == 0).all(dim=-1)  # Identify non-zero rows
    return [t[mask[i]] for i, t in enumerate(tensor)]

trimmed_tensors = remove_trailing_zeros(tensor_data)

# Step 2: Compute mean along dim=0
relevance_doc = torch.stack([t.mean(dim=0, keepdim=True) for t in trimmed_tensors])

relevance_doc.squeeze(1)


In [ ]:
import torch


# Compute cosine similarity for each query (2x14)
cosine_similarity = torch.nn.functional.cosine_similarity(db.unsqueeze(0), qv.unsqueeze(1), dim=2)

# Get top-k indices per query (2x5)
k = 5
topk_values, topk_indices = torch.topk(cosine_similarity, k, dim=1)

# Compute median similarity score for thresholding
median_values = torch.median(topk_values, dim=1, keepdim=True).values  # (2x1)

consine_threshold = 0.000
# Masks: Above median -> Relevant, Below median -> Non-relevant
relevant_mask = topk_values >= consine_threshold  # (2x5)
non_relevant_mask = topk_values < consine_threshold  # (2x5)

# Extract relevant and non-relevant indices
relevant_indices = topk_indices * relevant_mask  # Apply mask
non_relevant_indices = topk_indices * non_relevant_mask  # Apply mask

# Remove zero indices (needed when applying mask)
relevant_indices = relevant_indices[relevant_mask].reshape(qv.shape[0], -1)
non_relevant_indices = non_relevant_indices[non_relevant_mask].reshape(qv.shape[0], -1)

# Extract relevant and non-relevant vectors (2x(k/2)x3)
relevant_vecs = db[relevant_indices]
non_relevant_vecs = db[non_relevant_indices]

# Compute centroids (2x3)
if relevant_vecs.numel() > 0:  # Check if not empty
    centroid_relevant = relevant_vecs.mean(dim=1)
else:
    centroid_relevant = torch.zeros_like(qv[0])  # Match query vector size

if non_relevant_vecs.numel() > 0:  # Check if not empty
    centroid_non_relevant = non_relevant_vecs.mean(dim=1)
else:
    centroid_non_relevant = torch.zeros_like(qv[0])  # Match query vector size
centroid_relevant,centroid_non_relevant


In [ ]:
from src.utils.fun_retrieval import pseudo_relevance_feedback
a = pseudo_relevance_feedback(db,qv,CosSim_Thresh=0.96)
a